# SQL query from table names - Continued

In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

## The old Prompt

In [2]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [3]:
context = [ {'role':'system', 'content':"""
 CREATE SEVERAL (3+) TABLES HERE
"""} ]



In [4]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
WRITE IN YOUR CONTEXT QUERIES HERE
"""
})

In [5]:
#Function to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [6]:
#new
context_user = context.copy()
print(return_CCRMSQL("""YOUR QUERY HERE""", context_user))

I see that you have requested queries to be answered based on tables that have not been provided. Could you please provide the tables so that I can assist you with the queries you have in mind?


In [7]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("YOUR QUERY HERE", old_context_user))

This is your SQL:
```sql

Please provide a specific query to assist you.


In [8]:
#new
print(return_CCRMSQL("YOUR QUERY HERE", context_user))

I see you have not provided any tables for me to work with. Could you please provide the tables so that I can assist you with your queries?


In [9]:
#old
print(return_CCRMSQL("YOUR QUERY HERE", old_context_user))

This is your SQL:
```sql

Please provide a specific query to assist you.


# Exercise
 - Complete the prompts similar to what we did in class. 
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In [13]:
# Define the old prompt
old_context = [
    {'role': 'system', 'content': """
    You are a bot to assist in creating SQL commands. All your answers should start with 
    'This is your SQL,' followed by a SQL query. Use a simple SQL syntax.
    """},
    {'role': 'system', 'content': """
    First table:
    {
        "tableName": "employees",
        "fields": [
            {"name": "ID_usr", "type": "int"},
            {"name": "name", "type": "varchar"}
        ]
    }
    """},
    {'role': 'system', 'content': """
    Second table:
    {
        "tableName": "salary",
        "fields": [
            {"name": "ID_usr", "type": "int"},
            {"name": "year", "type": "date"},
            {"name": "salary", "type": "float"}
        ]
    }
    """},
    {'role': 'system', 'content': """
    Third table:
    {
        "tableName": "studies",
        "fields": [
            {"name": "ID", "type": "int"},
            {"name": "ID_usr", "type": "int"},
            {"name": "educational_level", "type": "int"},
            {"name": "Institution", "type": "varchar"},
            {"name": "Years", "type": "date"},
            {"name": "Speciality", "type": "varchar"}
        ]
    }
    """}
]

# Define the new prompt with a few-shot sample
new_context = [
    {'role': 'system', 'content': """
    CREATE TABLE employees (
        ID_usr INT PRIMARY KEY,
        name VARCHAR(50)
    );

    CREATE TABLE salary (
        ID_usr INT,
        year DATE,
        salary FLOAT,
        PRIMARY KEY(ID_usr, year)
    );

    CREATE TABLE studies (
        ID INT PRIMARY KEY,
        ID_usr INT,
        educational_level INT,
        Institution VARCHAR(100),
        Years DATE,
        Speciality VARCHAR(100)
    );

    -- Few-shot Examples:
    -- Question: "What is the salary of Martha in 2021?"
    -- SQL: SELECT salary FROM salary INNER JOIN employees ON salary.ID_usr = employees.ID_usr WHERE employees.name = 'Bob' AND year = '2021';

    -- Question: "List all employees and their educational institutions."
    -- SQL: SELECT employees.name, studies.Institution FROM employees INNER JOIN studies ON employees.ID_usr = studies.ID_usr;

    -- Question: "Find all employees with a salary above 50,000."
    -- SQL: SELECT employees.name FROM employees INNER JOIN salary ON employees.ID_usr = salary.ID_usr WHERE salary.salary > 50000;
    """}
]

# Sample queries to test both prompts
queries = [
    "Find employees who studied at LMU.",
    "Show the total salary paid to each employee in 2021.",
    "Which employees have the highest salary in 2021?"
]

# Test the queries using the old and new prompts
print("Testing Old Prompt:")
for query in queries:
    print(f"Query: {query}")
    print("Output:", return_CCRMSQL(query, old_context))
    print()

print("Testing New Prompt:")
for query in queries:
    print(f"Query: {query}")
    print("Output:", return_CCRMSQL(query, new_context))
    print()


Testing Old Prompt:
Query: Find employees who studied at LMU.
Output: This is your SQL query:

```sql
SELECT e.ID_usr, e.name
FROM employees e
JOIN studies s ON e.ID_usr = s.ID_usr
WHERE s.Institution = 'LMU';
```

Query: Show the total salary paid to each employee in 2021.
Output: This is your SQL query:

```sql
SELECT e.ID_usr, e.name, SUM(s.salary) AS total_salary_2021
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
WHERE YEAR(s.year) = 2021
GROUP BY e.ID_usr, e.name;
```

Query: Which employees have the highest salary in 2021?
Output: This is your SQL query:

```sql
SELECT e.ID_usr, e.name
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
WHERE s.year = '2021'
ORDER BY s.salary DESC
LIMIT 1;
```

Testing New Prompt:
Query: Find employees who studied at LMU.
Output: ```sql
SELECT employees.name
FROM employees
INNER JOIN studies ON employees.ID_usr = studies.ID_usr
WHERE studies.Institution = 'LMU';
```

Query: Show the total salary paid to each employee in 2021.
Output: ``


Findings
New Prompt Advantages: Improved accuracy due to clear schema definitions (CREATE TABLE) and example queries.
Reduced ambiguity in SQL command generation.

Old Prompt Issues: Model occasionally ignored certain tables or fields due to vague descriptions.


Challenges
Some queries failed when specific details (e.g., data types or constraints) were missing.
New prompt required more preparation effort (adding schemas and examples).

Lessons Learned
Few-shot learning significantly improves performance by anchoring the model to specific examples.
Clear, structured prompts help the model produce more accurate and valid SQL.